In [2]:
#Imports
import nfl_data_py as nfl
import pandas as pd

In [3]:
weekly_data_2019 = nfl.import_weekly_data([2019], downcast=True)
weekly_data_2019 = weekly_data_2019.loc[weekly_data_2019['season_type'] == 'REG']

# standardize team names
weekly_data_2019['recent_team'] = weekly_data_2019['recent_team'].replace({'LA': 'LAR'})

#drop irrelevant positions
relevent_positions = ['QB', 'RB', 'WR', 'TE']
weekly_data_2019 = weekly_data_2019.loc[weekly_data_2019['position'].isin(relevent_positions)]

#drop final week of season
weekly_data_2019 = weekly_data_2019.loc[weekly_data_2019['week'] != 17]

weekly_data_2019.head()

Downcasting floats.


,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2019,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,25.639999,25.639999
1,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2019,2,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,24.660000,24.660000
2,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2019,3,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,20.139999,20.139999
3,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2019,4,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,3.700000,3.700000
4,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2019,5,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,23.920000,23.920000


In [4]:
# calculate fantasy points and add to each row
weekly_data_2019['draftkings_fp'] = weekly_data_2019.apply(
    lambda row: round(
        row['passing_yards'] * 0.04 +
        row['passing_tds'] * 4 +
        row['interceptions'] * -1 +
        row['rushing_yards'] * 0.1 +
        row['rushing_tds'] * 6 +
        row['receiving_yards'] * 0.1 +
        row['receiving_tds'] * 6 +
        row['receptions'] * 1 +
        (row['receiving_fumbles_lost'] + row['rushing_fumbles_lost'] + row['sack_fumbles_lost'] ) * -1 +
        (row['passing_2pt_conversions'] + row['rushing_2pt_conversions'] + row['receiving_2pt_conversions']) * 2 +
        (3 if row['passing_yards'] >= 300 else 0) +
        (3 if row['rushing_yards'] >= 100 else 0) +
        (3 if row['receiving_yards'] >= 100 else 0),
        2
    ),
    axis=1
) 
weekly_data_2019.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,draftkings_fp
0,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2019,1,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,25.639999,25.639999,28.64
1,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2019,2,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,24.660000,24.660000,24.66
2,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2019,3,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,20.139999,20.139999,23.14
3,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2019,4,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,3.700000,3.700000,4.70
4,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2019,5,REG,...,NaN,0,NaN,NaN,NaN,NaN,0.0,23.920000,23.920000,27.92


In [5]:
# calculate drafkings points allowed by each defense to each position per week
defense_vs_pos = weekly_data_2019.groupby(['week', 'opponent_team', 'position'])['draftkings_fp'].sum().reset_index()
defense_vs_pos.rename(columns={'opponent_team': 'defense', 'draftkings_fp': 'dk_fp_allowed'}, inplace=True)
print(defense_vs_pos)

# calculate cumulative fantasy points allowed by each defense to each position
moving_avg_dvp = defense_vs_pos.copy()
moving_avg_dvp['mean_dk_points_allowed'] = moving_avg_dvp.groupby(['defense', 'position'])['dk_fp_allowed'].expanding().mean().reset_index(level=[0, 1], drop=True)
print(moving_avg_dvp)

# merge with player data
weekly_data_2019 = weekly_data_2019.merge(
    moving_avg_dvp[['week', 'defense', 'position', 'mean_dk_points_allowed']],
    left_on=['week', 'opponent_team', 'position'],
    right_on=['week', 'defense', 'position'],
    how='left'
)
weekly_data_2019.drop(columns=['defense'], inplace=True)
weekly_data_2019.rename(columns={'mean_dk_points_allowed': 'defense_vs_pos'}, inplace=True)
print(weekly_data_2019)

      week defense position  dk_fp_allowed
0        1     ARI       QB          31.60
1        1     ARI       RB          16.70
2        1     ARI       TE          30.60
3        1     ARI       WR          50.60
4        1     ATL       QB          14.32
...    ...     ...      ...            ...
1913    16     TEN       WR          39.30
1914    16     WAS       QB          38.28
1915    16     WAS       RB          46.90
1916    16     WAS       TE          21.50
1917    16     WAS       WR          53.20

[1918 rows x 4 columns]
      week defense position  dk_fp_allowed  mean_dk_points_allowed
0        1     ARI       QB          31.60               31.600000
1        1     ARI       RB          16.70               16.700000
2        1     ARI       TE          30.60               30.600000
3        1     ARI       WR          50.60               50.600000
4        1     ATL       QB          14.32               14.320000
...    ...     ...      ...            ...               

In [6]:
lines = nfl.import_sc_lines([2019])
lines.head()

,season,week,away_team,home_team,game_id,side,line
3072,2019,1,GB,CHI,2019090500,CHI,-3.0
3073,2019,1,GB,CHI,2019090500,GB,3.0
3074,2019,1,LA,CAR,2019090800,CAR,2.5
3075,2019,1,LA,CAR,2019090800,LA,-2.5
3076,2019,1,TEN,CLE,2019090801,CLE,-5.0
